In [ ]:
# default_exp feature_finding

# Feature Finding

> Functions related to feature finding

This part describes the implementation of the feature finding algorithm. The core of the algorithm is described in the [MaxQuant-Paper](https://www.nature.com/articles/nbt.1511).
The supplementary material explains the underlying methodology in great detail and is the foundation of the theoretical background that is described here.
A refined version of the algorithm was presented with [Dinosaur](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4933939/), which was also used as a reference for the Python implementation.

For the algorithm, we need serval modules:

1. Connecting Centroids to Hills

2. Refinement of Hills

3. Calculating Hill Statistics

4. Combining Hills to Isotope Patterns

5. Deconvolution of Isotope Patterns

## Loading Data

From the `IO` library, we already have `*.ms_data.hdf` container that contains centroided data. To use it in feature finding, we directly load the data.

## Connecting Centroids to Hills

### Connecting Centroids

In this step, we search for centroids that are connected (i.e., within a specific mass difference) over time. The connected centroids are termed hills. As we have a list of centroid lists, we ideally would have a list of tuples with indices to the centroids that describe the connected hills.

#### Pairs of Centroids
First, we define the function `get_pairs_centroids` that checks two lists of centroids and checks for pairs of centroids. As the masses in the centroid lists are sorted, we can loop through the centroids with two indices and compare the mass difference. If the mass difference is within the defined threshold, `ppm_tol` (default is 7ppm), we have found ourselves a pair. If no match is found, we can advance either index, depending on whether the mass difference is positive or negative. A pair is a `tuple` containing the index to the first centroid list `i` and the second centroid list `j` as well as the difference in ppm `delta`. We add all found pairs to a list termed `pairs`.

In [ ]:
#export
import alphapept.speed
import numpy as np
from alphapept.speed import parallel_compiled_func, cupy, jit_fun

@parallel_compiled_func
def connect_centroids_unidirection(idx, row_borders, connections, scores, centroids, max_gap, ppm_tol):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx

    for gap in range(max_gap + 1):
        y = x + gap + 1
        if y > row_borders.shape[0]:
            return

        start_index_f = 0
        if x > 0:
            start_index_f = row_borders[x - 1]

        centroids_1 = centroids[start_index_f: row_borders[x]]

        start_index_b = row_borders[y - 1]
        centroids_2 = centroids[start_index_b: row_borders[y]]

        i = 0
        j = 0

        while (i < len(centroids_1)) & (j < len(centroids_2)):
            mz1, mz2 = centroids_1[i], centroids_2[j]
            diff = mz1 - mz2
            mz_sum = mz1 + mz2
            delta = 2 * 1e6 * abs(diff) / mz_sum

            if delta < ppm_tol:

                if scores[x, i, gap] > delta:
                    scores[x, i, gap] = delta
                    connections[x, i, gap] = (connections.shape[1] * y) + j

            if diff > 0:
                j += 1
            else:
                i += 1

@parallel_compiled_func
def convert_connections_to_array(idx, from_r, from_c, to_r, to_c, row_borders, out_from_idx, out_to_idx):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx

    row = from_r[x]
    col = from_c[x]
    start_index_f = 0
    if row > 0:
        start_index_f = row_borders[row - 1]
    out_from_idx[x] = start_index_f + col

    row = to_r[x]
    col = to_c[x]
    start_index_f = 0
    if row > 0:
        start_index_f = row_borders[row - 1]
    out_to_idx[x] = start_index_f + col
    
@parallel_compiled_func
def eliminate_overarching_vertex(idx, from_idx, to_idx):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
    if x == 0:
        return

    if from_idx[x - 1] == from_idx[x]:
        to_idx[x] = -1
        
@parallel_compiled_func
def path_finder(idx, from_idx, to_idx, forward, backward):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
    fr = from_idx[x]
    to =  to_idx[x]

    forward[fr] = to
    backward[to] = fr

@parallel_compiled_func
def find_path_start(idx, forward, backward, path_starts):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx

    if forward[x] > -1 and backward[x] == -1:
        path_starts[x] = 0

@parallel_compiled_func
def find_path_length(idx, path_starts, forward, path_cnt):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx

    ctr = 1
    idx = path_starts[x]
    while forward[idx] > -1:
        ctr += 1
        idx = forward[idx]
    path_cnt[x] = ctr

@parallel_compiled_func
def fill_path_matrix(idx, path_start, forwards, out_hill_data, out_hill_ptr):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    path_position = 0
    idx = path_start[x]
    while idx > -1:
        out_hill_data[out_hill_ptr[x] + path_position] = idx
        idx = forwards[idx]
        path_position += 1

@parallel_compiled_func
def convert_to_coord_path(idx, out_rows, out_cols, connected_comps, d_width):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx

    idx = connected_comps[x]
    row = idx // d_width
    col = idx - row * d_width

    out_rows[x] = row
    out_cols[x] = col

#### Connecting all Centroids

As we now have a function that can find pairs of centroids when given two lists of centroids, the next step is to write a function that connects all the centroids within a dataset. For this, we define the function `connect_centroids`. Here, we use the `get_pairs_centroids` function to connect hills in consecutive scans `(n->n+1)` whenever their centroid m/z positions differ less than the defined threshold. If no matching peak is found in the next scan `(n+1)`, additional scans can be considered when looking in the scan after next `(n+2)`. The maximum distance between two hills can be set via the `gap` parameter. 

For an efficient implementation to find all connected hills over all scans, we create a sparse matrix, named `results` that represent indices to all possible centroids. As the number of centroids varies for each scan, not all positions within the matrix represent a valid centroid. To distinguish these values, we fill the matrix with `-1` upon initialization. We can use the matrix to store the matching centroid as an integer that encodes its position. The x,y index for each cell represents the indices to the centroid list. The idea here is to have an efficient data structure to represent connections between centroids.

To calculate indexes back and forth we define the functions `tup_to_ind` and `ind_to_tup`. This could in principle be also done with `numpy.unravel_index`, however, the numba implementation seems to be much faster.

Additionally, we define a sparse matrix named `scores` that is intended to store the mass difference. If there are multiple possible connections between two centroids, only the best (i.e., shortest distance) will be stored.

The idea behind the matrix-indexing is explained with the following example:

##### Example: 

Imagine you have three scans with the following centroids:

* Scan 0: 10, 20, 30
* Scan 1: 10.2, 40.1
* Scan 2: 40, 50, 60


When comparing consecutive scans and defining the maximum delta mass to be 0.5 find the following connections: (Scan No, Centroid No) -> (Scan No, Centroid No). As we cannot easily store tuples in the matrix, we convert tuple containing the position of the connected centroid to an integer.
* (0,0) -> (1,0) -> (3): 10 & 10.2 -> delta = 0.2
* (1,1) -> (2,0) -> (6): 40.1 & 40 -> delta = 0.1

Finally, we store this in the `results` matrix:

$\begin{bmatrix}
3 & -1  & -1 \\ 
-1 & 6 & -1\\ 
-1 & -1 & -1 
\end{bmatrix}$

The coressponding `scores` matrix will look as follows:

$\begin{bmatrix}
0.2 & -1  & -1 \\ 
-1 & 0.1 & -1\\ 
-1 & -1 & -1 
\end{bmatrix}$

This allows us to not only easily store connections between centroids but also perform a quick lookup for the delta of an existing connection. Note that it also only stores the best connection for each centroid. To extract the connected centroids, we can use `np.where(results >= 0)`. This implementation allows getting millions of connections within seconds. 

##### Foward and Backward Search

One thing that still needs to be considered is that when searching with a large mass tolerance, we could get multiple matches that we do not want. Consider the following example:

* Scan 0: 10 (a), 
* Scan 1: 10 (b) , 10.1 (c)
* Scan 2: 10 (d)

As we only allow one connection per mass, we would have the following connections:

* a -> b
* b -> d

Now if we have a large delta mass, e.g. of `0.1`, there would be an additional connection:

* c-> d


![undirected](images/undirected.png)

The resulting hill would have a branch to the c centroid, which is obviously intended. To circumvent this, we additionally perform the search backwards. (Note the functions `connect_centroids_forward` and `connect_centroids_backward`).  Now, as we only store, the best connection between two centroids  `d` will only be connected to `b`. To, therefore, remove branches, we remove all connections that are not double.


![directed](images/directed.png)

We define a wrapper function named `connect_centroids` that allows to search with a gaps ize and returns lists of indices of the connected centroids.

### Connecting hills

Having all the connected centroids, we would like to find `hills`, referring to centroids that are connected. We can achieve this by creating a graph `G` with the `networkx`-package. Here, each connection is added as an edge using the `add_edge` method to the graph. We can then use the `connected_components` function to extract all the connected components.

`NetworkX` is known to be memory [consuming](https://groups.google.com/forum/#!topic/networkx-discuss/Etd4GpkjPdA) and can take hundreds of bytes per edge. For millions of connections, we will, therefore, consume Gigabytes of memory. To keep the graph small, we continuously feed the graph until we reach a certain scan number `buffer size`. Once the limit is reached, we extract all completed hills and remove all nodes that are part of a completed hill or cannot be connected anymore. Once all edges are added, we perform a final extraction step. Finally, everything is put into the `get_hills` function.

#### Parallelization Strategies

Currently, the parallel connection of hills is not implemented yet. A possible way could be to process individual chunks of edges in parallel and then combine them together, as it is done in `Dinosaur`.

In [ ]:
#export

def find_centroid_connections(rowwise_peaks, row_borders, centroids, max_gap=2, ppm_tol=8):
    max_centroids = int(cupy.max(rowwise_peaks))
    spectra_cnt = len(row_borders) - 1

    connections = cupy.full((spectra_cnt, max_centroids, max_gap + 1), -1, dtype=np.int32)
    score = cupy.full((spectra_cnt, max_centroids, max_gap + 1), np.inf)
    
    

    connect_centroids_unidirection(row_borders,
                                   connections,
                                   score,
                                   centroids,
                                   max_gap,
                                   ppm_tol)
    
    score = score[cupy.where(score < np.inf)]
    
    score_median = cupy.median(score)
    score_std = cupy.std(score)
    
    
    del score, max_centroids, spectra_cnt

    c_shape = connections.shape
    from_r, from_c, from_g = cupy.where(connections >= 0)
    to_r = connections[from_r, from_c, from_g] // c_shape[1]
    to_c = connections[from_r, from_c, from_g] - to_r * c_shape[1]

    del connections, from_g
    return from_r, from_c, to_r, to_c, score_median, score_std

In [ ]:
#export 

def get_hills(centroids, from_idx, to_idx, min_hill_length=3):
    forward = cupy.full(centroids.shape[0], -1)
    backward = cupy.full(centroids.shape[0], -1)
    path_starts = cupy.full(centroids.shape[0], -1)

    path_finder(from_idx, to_idx, forward, backward)
    find_path_start(forward, backward, path_starts)

    # path_starts will now container the first index of all connected centroids
    path_starts = cupy.where(path_starts == 0)[0]

    path_node_cnt = cupy.full(path_starts.shape[0], -1)
    find_path_length(path_starts, forward, path_node_cnt)

    relavant_path_node = cupy.where(path_node_cnt >= min_hill_length)[0]
    path_starts = cupy.take(path_starts, relavant_path_node)
    path_node_cnt = cupy.take( path_node_cnt, relavant_path_node)
    del relavant_path_node

    # Generate the hill matix indice ptr data
    hill_ptrs = cupy.empty((path_starts.shape[0] + 1), dtype=cupy.int32)

    hill_ptrs[0] = 0
    hill_ptrs[1:] = path_node_cnt.cumsum()
    hill_data = cupy.empty((int(hill_ptrs[-1])), np.int32)

    fill_path_matrix(path_starts, forward, hill_data, hill_ptrs)

    del from_idx, to_idx, path_starts, forward, backward
    return hill_ptrs, hill_data, path_node_cnt

In [ ]:
#export
def connect_centroids(rowwise_peaks, row_borders, centroids, max_gap=2, ppm_tol=8):

    from_r, from_c, to_r, to_c, score_median, score_std = find_centroid_connections(rowwise_peaks,
                                                           row_borders,
                                                           centroids,
                                                           max_gap=max_gap,
                                                           ppm_tol=ppm_tol)

    from_idx = cupy.zeros(len(from_r), np.int32)
    to_idx = cupy.zeros(len(from_r), np.int32)

    convert_connections_to_array(from_r,
                                 from_c,
                                 to_r,
                                 to_c,
                                 row_borders,
                                 from_idx,
                                 to_idx)

    eliminate_overarching_vertex(from_idx, to_idx)

    relavent_idx = cupy.where(to_idx >= 0)
    from_idx = cupy.take(from_idx, relavent_idx)[0]
    to_idx = cupy.take(to_idx, relavent_idx)[0]

    del from_r, from_c, to_r, to_c, relavent_idx
    return from_idx, to_idx, score_median, score_std


In [ ]:
#export
def extract_hills(query_data, max_gap = 2, ppm_tol = 8):

    indices = cupy.array(query_data['indices_ms1'])
    mass_data = cupy.array(query_data['mass_list_ms1'])

    rowwise_peaks = indices[1:] - indices[:-1]
    row_borders = indices[1:]

    from_idx, to_idx, score_median, score_std = connect_centroids(rowwise_peaks, row_borders, mass_data, max_gap=max_gap, ppm_tol=ppm_tol)
    hill_ptrs, hill_data, path_node_cnt = get_hills(mass_data, from_idx, to_idx)

    del mass_data
    del indices
    
    if cupy.__name__ != 'numpy':
        hill_ptrs = hill_ptrs.get()
        hill_data = hill_data.get()
        path_node_cnt = path_node_cnt.get()
        
        score_median = score_median.get()
        score_std = score_std.get()

    
    return hill_ptrs, hill_data, path_node_cnt, score_median, score_std



### Hill Splitting
When having a hill with two or more maxima, we would like to split it at the minimum position. For this, we use a recursive approach. First, the minimum of a hill is detected. A hill is split at this minimum if the smaller of the surrounding maxima is at least the factor `split_level` larger than the minimum. For each split, the process is repeated.

In [ ]:
#export
@jit_fun
def fast_minima(y):
    minima = np.zeros(len(y))
    
    start = 0
    end = len(y)

    for i in range(start + 2, end - 2):
        if ((y[i - 1] > y[i]) & (y[i + 1] > y[i])) \
            or ((y[i - 1] > y[i]) & (y[i + 1] == y[i]) & (y[i + 2] > y[i])) \
            or ((y[i - 2] > y[i]) & (y[i - 1] == y[i]) & (y[i + 1] > y[i])) \
            or (((y[i - 2] > y[i]) & (y[i - 1] == y[i]) & (y[i + 1] == y[i]) & \
                (y[i + 2] > y[i]))):
            minima[i] = 1
            
    minima = minima.nonzero()[0]  
    
    return minima

@parallel_compiled_func(cpu_only = True)
def split(idx, hill_ptrs_range, hill_ptrs, int_data, hill_data, splits, split_level, window):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    k = hill_ptrs_range[x]
        
    start = hill_ptrs[k]
    end = hill_ptrs[k + 1]
    
    int_idx = hill_data[start:end] #index to hill data

    int_trace = int_data[int_idx]

    for i in range(len(int_idx)):
        min_index = max(0, i - window)
        max_index = min(len(int_idx), i + window + 1)
        int_trace[i] = np.median(int_trace[min_index:max_index])
        
    for i in range(len(int_idx)):
        min_index = max(0, i - window)
        max_index = min(len(int_idx), i + window + 1)
        int_trace[i] = np.mean(int_trace[min_index:max_index])

    #minima = (np.diff(np.sign(np.diff(int_trace))) > 0).nonzero()[0] + 1 #This works also but is slower
    
    minima = fast_minima(int_trace)
    
    sorted_minima = np.argsort(int_trace[minima])
    
    minima = minima[sorted_minima]
    
    for min_ in minima:

        minval = int_trace[min_]

        left_max = max(int_trace[:min_])
        right_max = max(int_trace[min_:])

        min_max = min(left_max, right_max)

        if (minval == 0) or ((min_max / minval) > split_level):
            splits[k] = start+min_
            break # Split only once per iteration
            
def split_hills(hill_ptrs, hill_data, int_data, split_level, window):
    splits = np.zeros(len(int_data), dtype=cupy.int32)
    to_check = np.arange(len(hill_ptrs)-1)
    
    while len(to_check) > 0:
        split(to_check, hill_ptrs, int_data, hill_data, splits, split_level, window)
        splitpoints = splits.nonzero()[0]

        to_check = np.zeros(len(hill_ptrs))
        to_check[splitpoints] = 1

        to_check = np.insert(to_check, splitpoints+1, np.ones(len(splitpoints))).nonzero()[0] #array, index, what
        hill_ptrs = np.insert(hill_ptrs, splitpoints+1, splits[splitpoints]) #array, index, what

        splits[:] = 0
        
    return hill_ptrs

### Filter Hills

To filter hills, we define a minimum length `hill_min_length`. All peaks below the threshold `hill_peak_min_length` are accepted as is. For longer hills, the intensity at the start and the end are compared to the maximum intensity. If the ratio of the maximum raw intensity to the smoothed intensity and the beginning and end are larger than `hill_peak_factor` the hills are accepted.

In [ ]:
#export
@parallel_compiled_func(cpu_only=True)
def check_large_hills(idx, large_peaks, hill_ptrs, hill_data, int_data, to_remove, large_peak = 40, hill_peak_factor = 2, window=1):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    k = large_peaks[x]

    start = hill_ptrs[k]
    end = hill_ptrs[k + 1]
    
    int_idx = hill_data[start:end] #index to hill data

    int_smooth_ = int_data[int_idx]

    for i in range(len(int_idx)):
        min_index = max(0, i - window)
        max_index = min(len(int_idx), i + window + 1)
        int_smooth_[i] = np.median(int_smooth_[min_index:max_index])
        
    for i in range(len(int_idx)):
        min_index = max(0, i - window)
        max_index = min(len(int_idx), i + window + 1)
        int_smooth_[i] = np.mean(int_smooth_[min_index:max_index])

    int_ = int_data[int_idx]

    max_ = np.max(int_)

    if (max_ / int_smooth_[0] > hill_peak_factor) & (max_ / int_smooth_[-1] > hill_peak_factor):
        to_remove[x] = 0 
    

def filter_hills(hill_data, hill_ptrs, int_data, large_peak =40, window = 1):
    
    large_peaks = np.where(np.diff(hill_ptrs)>=large_peak)[0]
    
    to_remove = np.ones(len(large_peaks), dtype=np.int32)
    check_large_hills(large_peaks, hill_ptrs, hill_data, int_data, to_remove, window)
    
    idx_ = np.ones(len(hill_data), dtype = np.int32)
    keep = np.ones(len(hill_ptrs)-1, dtype = np.int32)
    
    to_remove = to_remove.nonzero()[0]

    for _ in to_remove:
        idx_[hill_ptrs[_]:hill_ptrs[_+1]] = 0
        keep[_] = 0

    hill_lens = np.diff(hill_ptrs)
    keep_ = hill_lens[keep.nonzero()[0]]

    hill_data_ = hill_data[idx_.nonzero()[0]]
    hill_ptrs_ = np.empty((len(keep_) + 1), dtype=np.int32)
    hill_ptrs_[0] = 0
    hill_ptrs_[1:] = keep_.cumsum()    
    
    return hill_data_, hill_ptrs_
    

Since the mass estimate min the equation above is more complicated than just an average of the mj, a standard deviation based estimate of the error would not be appropriate. Therefore we calculate the error as a bootstrap2 estimate over B=150 bootstrap replications


## Calculating Hill Statistics

Next, we calculate summary statistics for the connected centroids. We can obtain a high precision mass estimate for each hill by taking the average of the the masses and weighting this by their intensiteis:

$$
\overline{m} = \frac{\sum_{j=1}^nm_jI_j}{\sum_{j=1}^nI_j}
$$

To estimate the mass error, we calculate the error as a boostrap estimate:
 
$$\Delta \overline{m} = \sqrt{\frac{\sum_{b=1}^{B}(\overline{m}_b - \overline{m} )}{(B-1)}}$$

The calculation of hill statistics for a single hill is implemented in `get_hill_stats`. To calculate the hill stats for a list of hills, we can call the wrapper `get_hill_data`.

In [ ]:
#export

@parallel_compiled_func(cpu_only=True) #Only CPU optimized at this point
def hill_stats(idx, hill_range, hill_ptrs, hill_data, int_data, mass_data, rt_, rt_idx, stats, hill_nboot_max, hill_nboot):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    np.random.seed(42)
        
    start = hill_ptrs[x]
    end = hill_ptrs[x + 1]

    idx_ = hill_data[start:end]

    int_ = int_data[idx_]
    mz_ = mass_data[idx_]

    int_sum = np.sum(int_) 
    int_area = np.trapz(rt_[rt_idx[idx_]], int_) #Area

    rt_min = rt_[rt_idx[idx_]].min()
    rt_max = rt_[rt_idx[idx_]].max()
    
    if len(idx_) > hill_nboot_max:
        bootsize = hill_nboot_max
    else:
        bootsize = len(idx_)

    averages = np.zeros(hill_nboot)
    average = 0

    for i in range(hill_nboot):
        boot = np.random.choice(len(int_), bootsize, replace=True)
        boot_mz = np.sum((mz_[boot] * int_[boot])) / np.sum(int_[boot])
        averages[i] = boot_mz
        average += boot_mz

    average_mz = average/hill_nboot

    delta = 0
    for i in range(hill_nboot):
        delta += (average_mz - averages[i]) ** 2 #maybe easier?
    delta_m = np.sqrt(delta / (hill_nboot - 1))
    
    stats[x,0] = average_mz
    stats[x,1] = delta_m
    stats[x,2] = int_sum
    stats[x,3] = int_area
    stats[x,4] = rt_min
    stats[x,5] = rt_max
    
def get_hill_data(query_data, hill_ptrs, hill_data, hill_nboot_max = 300, hill_nboot = 150):

    indices_ = np.array(query_data['indices_ms1'])
    rt_ = np.array(query_data['rt_list_ms1'])
    mass_data = np.array(query_data['mass_list_ms1'])
    scan_idx = np.searchsorted(indices_, np.arange(len(mass_data)), side='right') - 1
    int_data = np.array(query_data['int_list_ms1'])
    
    stats = np.zeros((len(hill_ptrs)-1, 6)) #mz, delta, rt_min, rt_max, sum_max
    hill_stats(np.arange(len(hill_ptrs)-1), hill_ptrs, hill_data, int_data, mass_data, rt_, scan_idx, stats, hill_nboot_max, hill_nboot)

    # sort the stats
    sortindex = np.argsort(stats[:,4]) #Sorted by rt_min
    stats = stats[sortindex,:]
    idxs_upper = stats[:,4].searchsorted(stats[:,5], side="right")
    sortindex_ = np.arange(len(sortindex))[sortindex]
    
    return stats, sortindex_, idxs_upper, scan_idx

## Combining Hills to Isotope Patterns

After obtaining summary statistics of hills, the next step is to check whether they belong together to form an isotope pattern. For this, we check wheter it is possible that they are neighbors in an isotope pattern, e.g. one having a 12C atom that has been replaced by a 13C version. The detailed criterion for the check is implemented in `check_isotope_pattern` and is as follows:


$$\left | \Delta m-\frac{\Delta M}{z} \right |\leq \sqrt{\left ( \frac{\Delta S}{z}  \right )^{2}+\Delta {m_{1}}^{2} +\Delta {m_{2}}^{2}}$$

The left side contains $\Delta m$, being the delta of the precise mass estimates from the summary statistics and $\Delta M = 1.00286864$, which is the mass difference ebtween the 13C peak and the monoisotopic peak in an averagine molecule of 1500 Da mass divided by the charge $z$.

The right side contains $\Delta S = 0.0109135$, which is the maximum shift that a sulphur atom can cause ($\Delta S = 2m(^{13}C) - 2m(^{12}C) - m(^{34}S) + m(^{32}S)$) and $\Delta {m_{1}}$ and $\Delta {m_{2}}$, which are the bootstrapped mass standard deviations.

In [ ]:
#export
from alphapept.constants import mass_dict
from numba import njit

DELTA_M = mass_dict['delta_M']
DELTA_S = mass_dict['delta_S']
maximum_offset = DELTA_M + DELTA_S

@njit
def check_isotope_pattern(mass1, mass2, delta_mass1, delta_mass2, charge, mass_range = 5):
    """
    Check if two masses could belong to the same isotope pattern
    """
    delta_mass1 = delta_mass1 * mass_range
    delta_mass2 = delta_mass2 * mass_range

    delta_mass = np.abs(mass1 - mass2)

    left_side = np.abs(delta_mass - DELTA_M / charge)
    right_side = np.sqrt((DELTA_S / charge) ** 2 + delta_mass1 ** 2 + delta_mass2 ** 2)

    return left_side <= right_side

In [ ]:
#hide
if False:
    def test_get_hill_data():
        test_centroids = [
            [(600, 1, 1, 1)],
            [(300, 1, 2, 2), (600, 0, 1, 2)],
            [(300, 10, 3, 3), (600, 20, 3, 3)],
            [(300, 20, 4, 4), (600, 40, 4, 4)],
            [(300, 40, 5, 5), (600, 80, 5, 5)],
            [(300, 20, 6, 6), (600, 40, 6, 6)],
            [(300, 10, 7, 7), (600, 20, 7, 7)],
            [(300, 1, 8, 8), (600, 0, 8, 8)],
        ]

        centroid_dtype = [("mz", float), ("int", float), ("scan_no", int), ("rt", float)]
        test_centroids_tmp = [np.array(_, dtype=centroid_dtype) for _ in test_centroids]

        test_centroids = List([_ for _ in test_centroids_tmp])

        test_hills = get_hills(test_centroids)
        sorted_hills, sorted_stats, sorted_data = get_hill_data(test_hills, test_centroids)

        # Check masses
        assert sorted_stats[0]["mz_avg"] == 600
        assert sorted_stats[1]["mz_avg"] == 300

        # Check intensities
        assert sorted_stats[0]["int_sum"] == 201
        assert sorted_stats[1]["int_sum"] == 102

        # Check start
        assert sorted_stats[0]["rt_min"] == 1
        assert sorted_stats[1]["rt_min"] == 2

    test_get_hill_data()

In [ ]:
charge = 1

mass1, delta_mass1 = 100, 0.1
mass2, delta_mass2 = 101.1, 0.05

print(check_isotope_pattern(mass1, mass2, delta_mass1, delta_mass2, charge))

mass2, delta_mass2 = 102.1, 0.05

print(check_isotope_pattern(mass1, mass2, delta_mass1, delta_mass2, charge))

True
False


### Cosine Correlation of two hills

An additional criterion that is being checked is that the intensity profiles have sufficient overalp in retention time. This is validated by ensuring that two hills have a cosine correlation of at least 0.6.

$$\frac{\sum_{s=s_{min}}^{s_{max}}I_sJ_s}{\sum_{s=s_{min}}^{s_{max}}I_s^{2} \sum_{s=s_{min}}^{s_{max}}J_s^{2}} \geq 0.6$$

The intensities of two hills are only compared if both have an intensity value in a particular scan. Otherwise, the intensity is set to zero. Additionally, an overlap of at least three elements is required. 

In [ ]:
#export

@njit
def correlate(scans_, scans_2, int_, int_2):
    
    min_one, max_one = scans_[0], scans_[-1]
    min_two, max_two = scans_2[0], scans_2[-1]

    if min_one + 3 > max_two:  # at least an overlap of 3 elements
        corr = 0
    elif min_two + 3 > max_one:
        corr = 0
    else:
        min_s = min(min_one, min_two)
        max_s = max(max_one, max_two)

        int_one_scaled = np.zeros(int(max_s - min_s + 1))
        int_two_scaled = np.zeros(int(max_s - min_s + 1))

        int_one_scaled[scans_ - min_s] = int_
        int_two_scaled[scans_2 - min_s] = int_2

        corr = np.sum(int_one_scaled * int_two_scaled) / np.sqrt(
            np.sum(int_one_scaled ** 2) * np.sum(int_two_scaled ** 2)
        )

    return corr

### Extracting pre-Isotope Patterns

Now having two criteria to check whether hills could, in principle, belong together, we define the wrapper functions `extract_edge` and `get_edges` to extract the connected hills. To minimize the number of comparisons we need to perform, we only compare the hills that overlap in time (i.e., the start of one hill `rt_min` needs to be before the end of the other hill `rt_max`) and are less than the sum of $\Delta M$ and $\Delta S$ apart. 

To extract all hills that belong together, we again rely on the `NetworkX`-package to extract the connected components. 

In [ ]:
#export
@njit
def extract_edge(stats, idxs_upper, runner, max_index, maximum_offset,  min_charge = 1, max_charge = 6, mass_range=5):
    edges = []

    mass1 = stats[runner, 0]
    delta_mass1 = stats[runner, 1]

    for j in range(runner+1, idxs_upper[runner]):
        mass2 = stats[j, 0]
        if np.abs(mass2 - mass1) <= maximum_offset:
            delta_mass2 = stats[j, 1]
            for charge in range(min_charge, max_charge + 1):
                if check_isotope_pattern(mass1, mass2, delta_mass1, delta_mass2, charge, mass_range):    
                    edges.append((runner, j))
                    break

    return edges

@parallel_compiled_func(cpu_only=True) #Only CPU optimized at this point
def edge_correlation(idx, to_keep, sortindex_, pre_edges, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    edge = pre_edges[x,:]
    
    y = sortindex_[edge[0]]
    start = hill_ptrs[y]
    end = hill_ptrs[y + 1]
    idx_ = hill_data[start:end]
    int_ = int_data[idx_]
    scans_ = scan_idx[idx_]

    con = sortindex_[edge[1]]
    start = hill_ptrs[con]
    end = hill_ptrs[con + 1]
    idx_2 = hill_data[start:end]
    int_2 = int_data[idx_2]
    scans_2 = scan_idx[idx_2]

    if correlate(scans_, scans_2, int_, int_2) > cc_cutoff:
        to_keep[x] = 1

In [ ]:
#export
import networkx as nx

def get_pre_isotope_patterns(stats, idxs_upper, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, maximum_offset, min_charge=1, max_charge=6, mass_range=5, cc_cutoff=0.6):
    pre_edges = []

    # Step 1
    for runner in range(len(stats)):
        pre_edges.extend(extract_edge(stats, idxs_upper, runner, idxs_upper[runner], maximum_offset, min_charge, max_charge, mass_range))

    to_keep = np.zeros(len(pre_edges), dtype='int')
    pre_edges = np.array(pre_edges)
    edge_correlation(to_keep, sortindex_, pre_edges, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff)
    edges = pre_edges[to_keep.nonzero()]

    G2 = nx.Graph()
    for i in range(len(edges)):
        G2.add_edge(edges[i][0], edges[i][1])

    pre_isotope_patterns = [
        sorted(list(c))
        for c in sorted(nx.connected_components(G2), key=len, reverse=True)
    ]
    
    return pre_isotope_patterns


### Extracting Isotope Patterns

The extracted pre-isotope patterns may not be consistent because their pair-wise mass differences may not correspond to the same charge. To extract isotope patterns from pre-isotope patterns, we need to ensure that they are consistent for a single charge. 

To do this, we start with the 100 most intense peaks from a pre-isotope pattern to be used as a seed. For each seed and charge we then try to extract the longest consistent isotope pattern. To check wheter a hill is consistent with the seed we employ a modified checking criterion (`check_isotope_pattern_directed`) to be as follows:

$$\left | m-m_j-\frac{j\Delta M}{z} \right |\leq \sqrt{\left ( \frac{\Delta S}{z}  \right )^{2}+\Delta {m}^{2} +\Delta {m_{j}}^{2}}$$

Here $m$ is the mass of a seed peak, and $m_{j}$ refers to a peak relative to the seed. $j$ refers to the peaks to the left or right (negative or positive index) within the pattern. $j$ needs to run over consecutive values so that gaps are not allowed. Besides this consistency check, two hills are also checked to have a cosine correlation of at least 0.6.

Programmatically, this is implemented in `grow_trail` and `grow`. These function uses a recursive approach that adds matching hills to the seed on the left and right side until no more hills can be added.

In [ ]:
#export

@njit
def check_isotope_pattern_directed(mass1, mass2, delta_mass1, delta_mass2, charge, index, mass_range):
    """
    Check if two masses could belong to the same isotope pattern

    """
    delta_mass1 = delta_mass1 * mass_range
    delta_mass2 = delta_mass2 * mass_range

    left_side = np.abs(mass1 - mass2 - index * DELTA_M / charge)
    right_side = np.sqrt((DELTA_S / charge) ** 2 + delta_mass1 ** 2 + delta_mass2 ** 2)

    return left_side <= right_side


@njit
def grow(trail, seed, direction, relative_pos, index, stats, pattern, charge, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff):
    """
    Grows isotope pattern based on a seed and direction

    """
    x = pattern[seed]  # This is the seed
    mass1 = stats[x,0]
    delta_mass1 = stats[x,1]
    
    k = sortindex_[x]
    start = hill_ptrs[k]
    end = hill_ptrs[k + 1]
    idx_ = hill_data[start:end]
    int_ = int_data[idx_]
    scans_ = scan_idx[idx_]

    growing = True

    while growing:
        if direction == 1:
            if seed + relative_pos == len(pattern):
                growing = False
                break
        else:
            if seed + relative_pos < 0:
                growing = False
                break

        y = pattern[seed + relative_pos]  # This is a reference peak
        
        l = sortindex_[y]
        
        mass2 = stats[y,0]
        delta_mass2 = stats[y,1]
        
        start = hill_ptrs[l]
        end = hill_ptrs[l + 1]
        idx_ = hill_data[start:end]
        int_2 = int_data[idx_]
        scans_2 = scan_idx[idx_]
        
        if correlate(scans_, scans_2, int_, int_2) > cc_cutoff:
            if check_isotope_pattern_directed(mass1, mass2, delta_mass1, delta_mass2, charge, -direction * index, mass_range):
                if direction == 1:
                    trail.append(y)
                else:
                    trail.insert(0, y)
                index += (
                    1
                )  # Greedy matching: Only one edge for a specific distance, will not affect the following matches

        delta_mass = np.abs(mass1 - mass2)

        if (delta_mass > (DELTA_M+DELTA_S) * index):  # the pattern is sorted so there is a maximum to look back
            break

        relative_pos += direction

    return trail

@njit
def grow_trail(seed, pattern, stats, charge, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff):
    """
    Wrapper to grow an isotope pattern to the left and right side
    """
    x = pattern[seed]
    trail = List()
    trail.append(x)
    trail = grow(trail, seed, -1, -1, 1, stats, pattern, charge, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff)
    trail = grow(trail, seed, 1, 1, 1, stats, pattern, charge, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff)

    return trail


@njit
def get_trails(seed, pattern, stats, charge_range, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff):
    """
    Wrapper to extract trails for a given charge range
    """
    trails = []
    for charge in charge_range:
        trail = grow_trail(seed, pattern, stats, charge, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff)

        trails.append(trail)

    return trails

In [ ]:
#export

def plot_pattern(pattern, sorted_hills, centroids, hill_data):

    """
    Helper function to plot a pattern
    """
    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10,10))
    centroid_dtype = [("mz", float), ("int", float), ("scan_no", int), ("rt", float)]

    mzs = []
    rts = []
    ints = []
    for entry in pattern:
        hill = sorted_hills[entry]
        hill_data = np.array([centroids[_[0]][_[1]] for _ in hill], dtype=centroid_dtype)

        int_profile = hill_data["int"]
        ax1.plot(hill_data["rt"], hill_data["int"])
        ax2.scatter(hill_data["rt"], hill_data["mz"], s = hill_data["int"]/5e5 )


    ax1.set_title('Pattern')
    ax1.set_xlabel('RT (min)')
    ax1.set_ylabel('Intensity')

    ax2.set_xlabel('RT (min)')
    ax2.set_ylabel('m/z')

    plt.show()

In [ ]:
#export

@njit
def get_minpos(y, split=5):
    """
    Function to get a list of minima in a trace.
    A minimum is returned if the ratio of lower of the surrounding maxima to the minimum is larger than the splitting factor.
    """
    minima = get_local_minima(y)
    minima_list = List()

    for minpos in minima:

        minval = y[minpos]
        left_side = y[:minpos]
        right_side = y[minpos:]

        left_max = np.max(left_side)
        right_max = np.max(right_side)

        minimum_max = np.min(np.array((left_max, right_max)))

        if minimum_max / minval > split:
            minima_list.append(minpos)

    return minima_list

@njit
def get_local_minima(y):
    """
    Function to return all local minima of a array
    """
    minima = List()
    for i in range(1, len(y) - 1):
        if is_local_minima(y, i):
            minima.append(i)
    return minima


@njit
def is_local_minima(y, i):
    return (y[i - 1] > y[i]) & (y[i + 1] > y[i])


#export
@njit
def truncate(array, intensity_profile, seedpos):
    """
    Function to truncate an intensity profile around its seedposition
    """
    minima = int_list_to_array(get_minpos(intensity_profile))

    if len(minima) > 0:
        left_minima = minima[minima < seedpos]
        right_minima = minima[minima > seedpos]

        # If the minimum is smaller than the seed
        if len(left_minima) > 0:
            minpos = left_minima[-1]
        else:
            minpos = 0

        if len(right_minima) > 0:
            maxpos = right_minima[0]
        else:
            maxpos = len(array)

        array = array[minpos:maxpos+1]

    return array, intensity_profile

## Isolating Isotope_patterns

The extraction of the longest consistent isotope pattern is implemented in `isolate_isotope_pattern`. Here, three additional checks for an isotope pattern are implemented. 

The first one is `truncate`. Here, one checks the seed position, whether it has a minimum to its left or right side. If a minimum is found, the isotope pattern is cut off at this position.

The second one is a mass filter. If the seed has a mass of smaller than 1000, the intensity maximum is detected, and all smaller masses are discarded. This reflects the averagine distribution for small masses where no minimum on the left side can be found.

The third one is `check_averagine` that relies on `pattern_to_mz` and `cosine_averagine`. It is used to ensure that the extracted isotope pattern has a cosine correlation of the averagine isotope pattern of the same mass of at least 0.6.

After the longest consistent isotope pattern is found, the hills are removed from the pre-isotope pattern, and the process is repeated until no more isotope patterns can be extracted from the pre-isotope patterns.

In [ ]:
#export
from alphapept.chem import mass_to_dist
from alphapept.constants import averagine_aa, isotopes

@njit
def check_averagine(stats, pattern, charge, averagine_aa, isotopes):

    masses, intensity = pattern_to_mz(stats, pattern, charge)

    spec_one = np.floor(masses).astype(np.int64)
    int_one = intensity

    spec_two, int_two = mass_to_dist(np.min(masses), averagine_aa, isotopes) # maybe change to no rounded version

    spec_two = np.floor(spec_two).astype(np.int64)

    return cosine_averagine(int_one, int_two, spec_one, spec_two)

@njit
def pattern_to_mz(stats, pattern, charge):
    """
    Function to calculate masses and intensities from pattern for a given charge
    """
    mzs = np.zeros(len(pattern))
    ints = np.zeros(len(pattern))

    for i in range(len(pattern)):
        entry = pattern[i]
        mzs[i] = mz_to_mass(stats[entry,0], charge)
        ints[i] = stats[entry,2]

    sortindex = np.argsort(mzs)

    masses = mzs[sortindex]
    intensity = ints[sortindex]

    return masses, intensity

@njit
def cosine_averagine(int_one, int_two, spec_one, spec_two):

    min_one, max_one = spec_one[0], spec_one[-1]
    min_two, max_two = spec_two[0], spec_two[-1]

    min_s = np.min(np.array([min_one, min_two]))
    max_s = np.max(np.array([max_one, max_two]))

    int_one_scaled = np.zeros(int(max_s - min_s + 1))
    int_two_scaled = np.zeros(int(max_s - min_s + 1))

    int_one_scaled[spec_one - min_s] = int_one
    int_two_scaled[spec_two - min_s] = int_two

    corr = np.sum(int_one_scaled * int_two_scaled) / np.sqrt(
        np.sum(int_one_scaled ** 2) * np.sum(int_two_scaled ** 2)
    )

    return corr



@njit
def int_list_to_array(numba_list):
    """
    Numba compatbilte function to convert a numba list with integers to a numpy array
    """
    array = np.zeros(len(numba_list), dtype=np.int64)

    for i in range(len(array)):

        array[i] = numba_list[i]

    return array

M_PROTON = mass_dict['Proton']

@njit
def mz_to_mass(mz, charge):
    """
    Function to calculate the mass from a mz value.
    """
    if charge < 0:
        raise NotImplementedError("Negative Charges not implemented.")

    mass = mz * charge - charge * M_PROTON

    return mass


@njit
def get_minpos(y, split=5):
    """
    Function to get a list of minima in a trace.
    A minimum is returned if the ratio of lower of the surrounding maxima to the minimum is larger than the splitting factor.
    """
    minima = get_local_minima(y)
    minima_list = List()

    for minpos in minima:

        minval = y[minpos]
        left_side = y[:minpos]
        right_side = y[minpos:]

        left_max = np.max(left_side)
        right_max = np.max(right_side)

        minimum_max = np.min(np.array((left_max, right_max)))

        if minimum_max / minval > split:
            minima_list.append(minpos)

    return minima_list

@njit
def get_local_minima(y):
    """
    Function to return all local minima of a array
    """
    minima = List()
    for i in range(1, len(y) - 1):
        if is_local_minima(y, i):
            minima.append(i)
    return minima


@njit
def is_local_minima(y, i):
    return (y[i - 1] > y[i]) & (y[i + 1] > y[i])


#export
@njit
def truncate(array, intensity_profile, seedpos):
    """
    Function to truncate an intensity profile around its seedposition
    """
    minima = int_list_to_array(get_minpos(intensity_profile))

    if len(minima) > 0:
        left_minima = minima[minima < seedpos]
        right_minima = minima[minima > seedpos]

        # If the minimum is smaller than the seed
        if len(left_minima) > 0:
            minpos = left_minima[-1]
        else:
            minpos = 0

        if len(right_minima) > 0:
            maxpos = right_minima[0]
        else:
            maxpos = len(array)

        array = array[minpos:maxpos+1]
        intensity_profile = intensity_profile[minpos:maxpos+1]

    return array, intensity_profile

In [ ]:
#hide

if False:
    def test_get_minpos():
        """
        Generate an intensity profile with local minima
        Check that the minima are found

        """
        intensity_profile = np.ones(20) * 10

        minima_ref = [3, 7, 10, 17]

        for minimum in minima_ref:
            intensity_profile[minimum] = 1

        minima = get_minpos(intensity_profile)

        minima_list = [_ for _ in minima]

        assert minima_list == minima_ref

    test_get_minpos() 

    def test_truncate():
        """
        Generate an intensity profile with local minima
        Check wheter the the profile is correctly truncated with respect to the seed

        """
        array = np.arange(0, 20)
        intensity_profile = np.ones(20) * 10

        minima_ref = [3, 7, 10, 17]

        for minimum in minima_ref:
            intensity_profile[minimum] = 1

        seedpos = 5
        truncated = truncate(array, intensity_profile, seedpos)
        assert np.all(truncated == np.array([3, 4, 5, 6, 7]))

        seedpos = 0
        truncated = truncate(array, intensity_profile, seedpos)
        assert np.all(truncated == np.array([0, 1, 2, 3]))

        seedpos = len(array)
        truncated = truncate(array, intensity_profile, seedpos)
        assert np.all(truncated == np.array([17, 18, 19]))

    test_truncate()

## Isotope Patterns

The wrapper function `get_isotope_patterns` iterates over all pre_isotope_patterns.

In [ ]:
#export
@njit
def isolate_isotope_pattern(pre_pattern, hill_ptrs, hill_data, int_data, scan_idx, stats, sortindex_, mass_range, charge_range, averagine_aa, isotopes, seed_masses, cc_cutoff):
    """
    Isolate isotope patterns
    """

    longest_trace = 0
    champion_trace = None
    champion_charge = 0

    # Sort patterns by mass
    
    sortindex = np.argsort(stats[pre_pattern][:,0]) #intensity
    sorted_pattern = pre_pattern[sortindex]
    massindex = np.argsort(stats[sorted_pattern][:,2])[::-1][:seed_masses]

    # Use all the elements in the pre_pattern as seed

    for seed in massindex:  # Loop through all seeds
        seed_global = sorted_pattern[seed]

        trails = get_trails(seed, sorted_pattern, stats, charge_range, mass_range, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, cc_cutoff)

        for index, trail in enumerate(trails):
            if len(trail) > longest_trace:  # Needs to be longer than the current champion

                arr = int_list_to_array(trail)
                intensity_profile = stats[arr][:,2]
                seedpos = np.nonzero(arr==seed_global)[0][0]

                # truncate around the seed...
                arr, intensity_profile = truncate(arr, intensity_profile, seedpos)

                # Remove lower masses:
                # Take the index of the maximum and remove all masses on the left side
                if charge_range[index] * stats[seed_global, 0] < 1000:
                    maxpos = np.argmax(intensity_profile)
                    arr = arr[maxpos:]

                if len(arr) > longest_trace:
                    # Averagine check
                    cc = check_averagine(stats, arr, charge_range[index], averagine_aa, isotopes)
                    if cc > 0.6:
                        # Update the champion
                        champion_trace = arr
                        champion_charge = charge_range[index]
                        longest_trace = len(arr)
                        
    return champion_trace, champion_charge

In [ ]:
if False:
    def test_get_isotope_patterns():

        test_centroids = [
            [
                (300, 50, 1, 1),
                (300.501, 40, 1, 1),
                (301.003, 30, 1, 1),
                (301.504, 20, 1, 1),
                (302.006, 10, 1, 1),
            ],
            [
                (300, 50, 2, 2),
                (300.501, 40, 2, 2),
                (301.003, 30, 2, 2),
                (301.504, 20, 2, 2),
                (302.006, 10, 2, 2),
            ],
            [
                (300, 50, 3, 3),
                (300.501, 40, 3, 3),
                (301.003, 30, 3, 3),
                (301.504, 20, 3, 3),
                (302.006, 10, 3, 3),
            ],
            [
                (300, 50, 4, 4),
                (300.501, 40, 4, 4),
                (301.003, 30, 4, 4),
                (301.504, 20, 4, 4),
                (302.006, 10, 4, 4),
            ],
            [
                (300, 50, 5, 5),
                (300.501, 40, 5, 5),
                (301.003, 30, 5, 5),
                (301.504, 20, 5, 5),
                (302.006, 10, 5, 5),
            ],
            [(400, 10, 6, 6), (401, 10, 6, 6), (402, 10, 6, 6)],
            [(400, 10, 7, 7), (401, 10, 7, 7), (402, 10, 7, 7)],
            [(400, 10, 8, 8), (401, 10, 8, 8), (402, 10, 8, 8)],
            [(400, 10, 9, 9), (401, 10, 9, 9), (402, 10, 9, 9)],
        ]

        centroid_dtype = [("mz", float), ("int", float), ("scan_no", int), ("rt", float)]
        test_centroids_tmp = [np.array(_, dtype=centroid_dtype) for _ in test_centroids]

        test_centroids = List([_ for _ in test_centroids_tmp])

        test_hills = get_hills(test_centroids)
        sorted_hills, stats, data = get_hill_data(test_hills, test_centroids)
        pre_patterns = get_edges(stats, data)

        isotope_patterns, isotope_charges = get_isotope_patterns(pre_patterns, stats, data, averagine_aa, isotopes)
        assert np.all(isotope_patterns[0] == np.array([0, 1, 2, 3, 4]))
        assert isotope_charges[0] == 2
        assert np.all(isotope_patterns[1] == np.array([5,6,7]))
        assert isotope_charges[1] == 1

    test_get_isotope_patterns()

In [ ]:
#export

from numba.typed import List

def get_isotope_patterns(pre_isotope_patterns, hill_ptrs, hill_data, int_data, scan_idx, stats, sortindex_,  averagine_aa, isotopes, min_charge = 1, max_charge = 6, mass_range = 5, seed_masses = 100, cc_cutoff=0.6, callback=None):
    """
    Wrapper function to iterate over pre_isotope_patterns
    """

    isotope_patterns = []
    isotope_charges = []

    charge_range = List()

    for i in range(min_charge, max_charge + 1):
        charge_range.append(i)

    isotope_patterns = []
    isotope_charges = []

    for idx, pre_pattern in enumerate(pre_isotope_patterns):
        extract = True
        while extract:
            isotope_pattern, isotope_charge = isolate_isotope_pattern(np.array(pre_pattern), hill_ptrs, hill_data, int_data, scan_idx, stats, sortindex_, mass_range, charge_range, averagine_aa, isotopes, seed_masses, cc_cutoff)
            if isotope_pattern is None:
                length = 0
            else:
                length = len(isotope_pattern)

            if length > 1:
                isotope_charges.append(isotope_charge)
                isotope_patterns.append(isotope_pattern)

                pre_pattern = [_ for _ in pre_pattern if _ not in isotope_pattern]

                if len(pre_pattern) <= 1:
                    extract = False
            else:
                extract = False


        if callback:
            callback((idx+1)/len(pre_isotope_patterns))
            
            
    iso_patterns = np.zeros(sum([len(_) for _ in isotope_patterns]), dtype=np.int64)
    
    iso_idx = np.zeros(len(isotope_patterns)+1, dtype='int')
    
    
    start = 0
    for idx, _ in enumerate(isotope_patterns):
        iso_patterns[start:start+len(_)] = _
        start += len(_)
        iso_idx[idx+1] = start
    
        

    return iso_patterns, iso_idx, np.array(isotope_charges)

In [ ]:
#export
@parallel_compiled_func(cpu_only=True)
def report_(idx, isotope_charges, isotope_patterns, iso_idx, stats, sortindex_, hill_ptrs, hill_data, int_data, rt_, rt_idx, results):
    if idx == -1:
        x = alphapept.speed.cuda.grid(1)
    else:
        x = idx
        
    i = x
        
    pattern = isotope_patterns[iso_idx[i]:iso_idx[i+1]]
    isotope_data = stats[pattern]

    mz = np.min(isotope_data[:, 0])
    mz_std = np.mean(isotope_data[:, 1])
    charge = isotope_charges[i]
    mass = mz_to_mass(mz, charge)
    int_max_idx = np.argmax(isotope_data[:, 2])
    mz_most_abundant = isotope_data[:, 0][int_max_idx]
    
    int_max = isotope_data[:,2][int_max_idx]

    rt_start = isotope_data[int_max_idx, 4] # This is the start of the most abundant trace
    rt_end = isotope_data[int_max_idx, 5]

    int_sum = np.sum(isotope_data[:, 2])
    
    # better measurement of the peak with interpolation
    
    rt_min_ = min(isotope_data[:, 4])
    rt_max_ = max(isotope_data[:, 5])
    
    rt_range = np.linspace(rt_min_, rt_max_, 100)
    trace_sum = np.zeros_like(rt_range)
    
    for k in pattern:
        x = sortindex_[k]
        
        start = hill_ptrs[x]
        end = hill_ptrs[x + 1]
        idx_ = hill_data[start:end]
        int_ = int_data[idx_]        
        rts = rt_[rt_idx[idx_]]

        interpolation = np.interp(rt_range, rts, int_)
        
        #Filter 
        
        interpolation[:(rt_range < rts[0]).sum()] = 0
        
        right_cut = (rt_range > rts[-1]).sum()
        if right_cut > 0:
            interpolation[-right_cut:]= 0 
        
        trace_sum += interpolation
        

        #plt.plot(rt_range, interpolation)

            
    rt_apex_idx = trace_sum.argmax()
    rt_apex = rt_range[rt_apex_idx]
    
    trace = trace_sum
    half_max = trace.max()/2
    
    if rt_apex_idx == 0:
        left_apex = 0
    else:
        left_apex = np.abs(trace[:rt_apex_idx]-half_max).argmin()
    right_apex = np.abs(trace[rt_apex_idx:]-half_max).argmin()+rt_apex_idx
    
    int_apex = trace_sum[rt_apex_idx]

    fwhm = rt_range[right_apex] - rt_range[left_apex]
    
    n_isotopes = len(pattern)

    rt_cutoff = 0.95 #5%
    if rt_apex_idx == 0:
        rt_min_idx = 0
    else:
        rt_min_idx = np.abs(trace[:rt_apex_idx]-trace.max()*(1-rt_cutoff)).argmin()
    rt_max_idx = np.abs(trace[rt_apex_idx:]-trace.max()*(1-rt_cutoff)).argmin()+rt_apex_idx

    #plt.xlabel('rt')
    #plt.ylabel('int')
    #plt.show()
    #plt.plot(rt_range, trace_sum)

    #plt.plot([rt_range[left_apex], rt_range[right_apex]], [(trace[left_apex] + trace[right_apex])/2]*2, 'k:')

    #plt.plot(rt_range[rt_apex_idx], trace[rt_apex_idx], 'k*')
    #plt.plot(rt_range[rt_min_idx], trace[rt_min_idx], 'k*')
    #plt.plot(rt_range[rt_max_idx], trace[rt_max_idx], 'k*')

    #plt.show()
            
    rt_start = rt_range[rt_min_idx]
    rt_end = rt_range[rt_max_idx]
    
    
    
    
    int_area = np.trapz(trace_sum[rt_min_idx:rt_max_idx], rt_range[rt_min_idx:rt_max_idx])
    int_sum = trace_sum.sum()
        
    results[i,:] = np.array([mz, mz_std, mz_most_abundant, charge, rt_start, rt_apex, rt_end, fwhm, n_isotopes, mass, int_apex, int_area, int_sum])
           
    

In [ ]:
#export
import pandas as pd

def feature_finder_report(query_data, isotope_patterns, isotope_charges, iso_idx, stats, sortindex_, hill_ptrs, hill_data,):
    rt_ = np.array(query_data['rt_list_ms1'])
    indices_ = np.array(query_data['indices_ms1'])
    mass_data = np.array(query_data['mass_list_ms1'])
    rt_idx = np.searchsorted(indices_, np.arange(len(mass_data)), side='right') - 1
    
    int_data = np.array(query_data['int_list_ms1'])

    results = np.zeros((len(isotope_charges), 13))

    report_(isotope_charges, isotope_patterns, iso_idx, stats, sortindex_, hill_ptrs, hill_data, int_data, rt_, rt_idx, results)

    df = pd.DataFrame(results, columns = ['mz','mz_std','mz_most_abundant','charge','rt_start','rt_apex','rt_end','fwhm','n_isotopes','mass','int_apex','int_area', 'int_sum'])

    df.sort_values(['rt_start','mz'])
    
    return df


## Data Output

For each feature that is found we extract summary statistics and put it in tabular form to be used as as pandas dataframe.

## Plotting

For quality control reasons we also employ a function to plot a feature in its local environment.

In [ ]:
#export
def plot_isotope_pattern(index, df, sorted_stats, centroids, scan_range=100, mz_range=2, plot_hills = False):
    """
    Plot an isotope pattern in its local environment
    """

    markersize = 10
    plot_offset_mz = 1
    plot_offset_rt = 2

    feature =  df.loc[index]

    scan = rt_dict[feature['rt_apex']]

    start_scan = scan-scan_range
    end_scan = scan+scan_range

    mz_min = feature['mz']-mz_range-plot_offset_mz
    mz_max = feature['mz']+mz_range+plot_offset_mz

    sub_data = np.hstack(centroids[start_scan:end_scan])

    selection = sub_data[(sub_data['mz']>mz_min) & (sub_data['mz']<mz_max)]

    min_rt = selection['rt'].min() - plot_offset_rt
    max_rt = selection['rt'].max() + plot_offset_rt

    hill_selection = sorted_stats[(sorted_stats['mz_avg']>mz_min) & (sorted_stats['mz_avg']<mz_max) & (sorted_stats['rt_max']<max_rt) & (sorted_stats['rt_min']>min_rt)]

    plt.style.use('dark_background')

    plt.figure(figsize=(15,15))
    plt.scatter(selection['rt'], selection['mz'], c= np.log(selection['int']), marker='s', s=markersize, alpha=0.9)
    plt.colorbar()
    plt.grid(False)
    plt.xlabel('RT (min)')
    plt.ylabel('m/z')

    box_height = mz_range/50

    if plot_hills:
        for hill in hill_selection:
            bbox = [hill['rt_min'], hill['mz_avg']-box_height, hill['rt_max'], hill['mz_avg']+box_height]

            rect = plt.Rectangle((bbox[0], bbox[1]),
                                      bbox[2] - bbox[0],
                                      bbox[3] - bbox[1], fill=False,
                                      edgecolor='w', linewidth=1, alpha = 0.3)
            plt.gca().add_patch(rect)


    feature_selection = df[(df['mz']>mz_min) & (df['mz']<mz_max) & (df['rt_end']<max_rt) & (df['rt_start']>min_rt)]

    for f_idx in feature_selection.index:
        for c_idx in range(len(sorted_stats[isotope_patterns[f_idx]])-1):

            start = sorted_stats[isotope_patterns[f_idx]][c_idx]
            end = sorted_stats[isotope_patterns[f_idx]][c_idx+1]

            start_mass = start['mz_avg']
            start_rt = (start['rt_min']+start['rt_max'])/2

            end_mass = end['mz_avg']
            end_rt = (end['rt_min']+end['rt_max'])/2

            plt.plot([start_rt, end_rt], [start_mass, end_mass], '+', color='y')
            plt.plot([start_rt, end_rt], [start_mass, end_mass], ':', color='y')

        if plot_hills:
            for hill_idx in isotope_patterns[f_idx]:

                hill = sorted_stats[hill_idx]
                bbox = [hill['rt_min'], hill['mz_avg']-box_height, hill['rt_max'], hill['mz_avg']+box_height]

                rect = plt.Rectangle((bbox[0], bbox[1]),
                                          bbox[2] - bbox[0],
                                          bbox[3] - bbox[1], fill=False,
                                          edgecolor='g', linewidth=1, alpha = 0.8)
                plt.gca().add_patch(rect)


    plt.xlim([min_rt+plot_offset_rt, max_rt-plot_offset_rt])
    plt.ylim([mz_min+plot_offset_mz, mz_max-plot_offset_mz])
    plt.title('Pattern')
    plt.show()

    plt.style.use('ggplot')

## External Feature Finder

To utilize the command-line Feature Finder from Bruker `4DFF-3.13` - `uff-cmdline2.exe`, we call it via a subprocess and wait until completion.

In [ ]:
#export
import subprocess
import os
import platform 


def extract_bruker(file, base_dir = "ext/bruker/FF", config = "proteomics_4d.config"):
    """
    Call Bruker Feautre Finder via subprocess
    """

    feature_path = file + '/'+ os.path.split(file)[-1] + '.features'

    base_dir = os.path.join(os.path.dirname(__file__), base_dir)
    
    if os.path.exists(feature_path):
        return feature_path
    else:
        
        operating_system = platform.system()
        
        if operating_system == 'Linux':
            ff_dir = os.path.join(base_dir, 'linux64','uff-cmdline2')
            logging.info('Using Linux FF')
        elif operating_system == 'Windows':
            ff_dir = os.path.join(base_dir, 'win64','uff-cmdline2.exe')
            logging.info('Using Windows FF')
        else:
            raise NotImplementedError(f"System {operating_system} not supported.")
        
        if not os.path.isfile(ff_dir):
            raise FileNotFoundError(f'Bruker feature finder cmd not found here {ff_dir}.')

        config_path = base_dir + '/'+ config

        if not os.path.isfile(config_path):
            raise FileNotFoundError(f'Config file not found here {config_path}.')

        if operating_system == 'Windows':
            FF_parameters = [ff_dir,'--ff 4d',f'--readconfig "{config_path}"', f'--analysisDirectory "{file}"']

            process = subprocess.Popen(' '.join(FF_parameters), stdout=subprocess.PIPE)
            for line in iter(process.stdout.readline, b''):
                logtxt = line.decode('utf8')
                logging.info(logtxt[48:].rstrip()) #Remove logging info from FF
        elif operating_system == 'Linux':
            FF_parameters = [
                ff_dir,
                '--ff',
                '4d',
                '--readconfig',
                config_path,
                '--analysisDirectory',
                file
            ]
            process = subprocess.run(FF_parameters, stdout=subprocess.PIPE)

        if os.path.exists(feature_path):
            return feature_path
        else:
            raise FileNotFoundError(f"Feature file {feature_path} does not exist.")


import sqlalchemy as db

def convert_bruker(feature_path):
    """
    Reads feature table and converts to feature table to be used with AlphaPept

    """
    engine_featurefile = db.create_engine('sqlite:///{}'.format(feature_path))
    feature_table = pd.read_sql_table('LcTimsMsFeature', engine_featurefile)

    from alphapept.constants import mass_dict

    M_PROTON = mass_dict['Proton']
    feature_table['Mass'] = feature_table['MZ'].values * feature_table['Charge'].values - feature_table['Charge'].values*M_PROTON
    feature_table = feature_table.rename(columns={"MZ": "mz","Mass": "mass", "RT": "rt_apex", "RT_lower":"rt_start", "RT_upper":"rt_end", "Mobility": "mobility", "Mobility_lower": "mobility_lower", "Mobility_upper": "mobility_upper", "Charge":"charge","Intensity":'int_sum'})
    feature_table['rt_apex'] = feature_table['rt_apex']/60
    feature_table['rt_start'] = feature_table['rt_start']/60
    feature_table['rt_end'] = feature_table['rt_end']/60

    return feature_table


def map_bruker(feature_path, feature_table, query_data):
    """
    Map Ms1 to Ms2 via Table FeaturePrecursorMapping from Bruker FF
    """
    engine_featurefile = db.create_engine('sqlite:///{}'.format(feature_path))

    mapping = pd.read_sql_table('FeaturePrecursorMapping', engine_featurefile)
    mapping = mapping.set_index('PrecursorId')
    feature_table= feature_table.set_index('Id')


    query_prec_id = query_data['prec_id']

    #Now look up the feature for each precursor

    mass_matched = []
    mz_matched = []
    rt_matched = []
    query_idx = []
    f_idx = []

    for idx, prec_id in tqdm(enumerate(query_prec_id)):
        try:
            f_id = mapping.loc[prec_id]['FeatureId']
            all_matches = feature_table.loc[f_id]
            if type(f_id) == np.int64:
                match = all_matches
                mz_matched.append(match['mz'])
                rt_matched.append(match['rt_apex'])
                mass_matched.append(match['mass'])
                query_idx.append(idx)
                f_idx.append(match['FeatureId'])

            else:
                for k in range(len(all_matches)):
                    match = all_matches.iloc[k]
                    mz_matched.append(match['mz'])
                    rt_matched.append(match['rt_apex'])
                    mass_matched.append(match['mass'])
                    query_idx.append(idx)
                    f_idx.append(match['FeatureId'])

        except KeyError:
            pass

    features = pd.DataFrame(np.array([mass_matched, mz_matched, rt_matched, query_idx, f_idx]).T, columns = ['mass_matched', 'mz_matched', 'rt_matched', 'query_idx', 'feature_idx'])

    features['query_idx'] = features['query_idx'].astype('int')

    return features

## Wrapper

In [ ]:
#export
import numpy as np

import logging
import os
from alphapept.search import query_data_to_features
import alphapept.io
import functools


def find_features(to_process, callback = None, parallel = False):
    """
    Wrapper for feature finding
    """
    try:
        index, settings = to_process
        file_name = settings['experiment']['file_paths'][index]

        base, ext = os.path.splitext(file_name)
        
        if ext.lower() == '.raw':
            datatype='thermo'
        elif ext.lower() == '.d':
            datatype='bruker'
        else:
            raise NotImplementedError('File extension {} not understood.'.format(ext))

        out_file = f"{base}.ms_data.hdf"
         
        skip = True
        if os.path.isfile(out_file):
            try:
                alphapept.io.MS_Data_File(
                    out_file
                ).read(dataset_name="features")
                logging.info(
                    'Found *.hdf with features for {}'.format(out_file)
                )
            except KeyError:

                logging.info(
                    'No *.hdf file with features found for {}. Adding to feature finding list.'.format(out_file)
                )
                skip = False
                
        if not skip:
            ms_file = alphapept.io.MS_Data_File(out_file, is_read_only=False)
            query_data = ms_file.read_DDA_query_data()

            if not settings['workflow']["find_features"]:
                features = query_data_to_features(query_data)
            else:
                if datatype == 'thermo':
                    
                    from alphapept.constants import averagine_aa, isotopes
                    
                    f_settings = settings['features']
                    max_gap = f_settings['max_gap']
                    ppm_tol = f_settings['ppm_tol']
                    split_level = f_settings['split_level']
                    window = f_settings['smoothing_window']
                    large_peak = f_settings['large_peak']
                    
                    min_charge = f_settings['min_charge']
                    max_charge = f_settings['max_charge']
                    seed_masses = f_settings['seed_masses']
                    
                    hill_nboot_max = f_settings['hill_nboot_max']
                    hill_nboot = f_settings['hill_nboot']
                    
                    mass_range = f_settings['mass_range']
                    
                    min_correlation = f_settings['min_correlation']
                    
                    logging.info('Feature finding on {}'.format(file_name))
                    hill_ptrs, hill_data, path_node_cnt, score_median, score_std = extract_hills(query_data, max_gap = max_gap, ppm_tol = ppm_tol)
                    logging.info(f'Number of hills {len(hill_ptrs):,}, len = {np.mean(path_node_cnt):.2f}')
                    
                    logging.info(f'Repeating hill extraction with ppm_tol {score_median+score_std*3:.2f}')
                    
                    hill_ptrs, hill_data, path_node_cnt, score_median, score_std = extract_hills(query_data, max_gap = max_gap, ppm_tol = score_median+score_std*3)
                    logging.info(f'Number of hills {len(hill_ptrs):,}, len = {np.mean(path_node_cnt):.2f}')

                    int_data = np.array(query_data['int_list_ms1'])

                    hill_ptrs = split_hills(hill_ptrs, hill_data, int_data, split_level=split_level, window = window) #hill lenght is inthere already
                    logging.info(f'After split hill_ptrs {len(hill_ptrs):,}')

                    hill_data, hill_ptrs = filter_hills(hill_data, hill_ptrs, int_data, large_peak =large_peak, window=window)

                    logging.info(f'After filter hill_ptrs {len(hill_ptrs):,}')

                    stats, sortindex_, idxs_upper, scan_idx = get_hill_data(query_data, hill_ptrs, hill_data, hill_nboot_max = hill_nboot_max, hill_nboot = hill_nboot)
                    logging.info('Extracting hill stats complete')

                    pre_isotope_patterns = get_pre_isotope_patterns(stats, idxs_upper, sortindex_, hill_ptrs, hill_data, int_data, scan_idx, maximum_offset, min_charge=min_charge, max_charge=max_charge, mass_range=mass_range, cc_cutoff=min_correlation)
                    logging.info('Found {:,} pre isotope patterns.'.format(len(pre_isotope_patterns)))

                    isotope_patterns, iso_idx, isotope_charges = get_isotope_patterns(pre_isotope_patterns, hill_ptrs, hill_data, int_data, scan_idx, stats, sortindex_, averagine_aa, isotopes, min_charge = min_charge, max_charge = max_charge, mass_range = mass_range, seed_masses = seed_masses, cc_cutoff = min_correlation, callback=None)
                    logging.info('Extracted {:,} isotope patterns.'.format(len(isotope_charges)))

                    feature_table = feature_finder_report(query_data, isotope_patterns, isotope_charges, iso_idx, stats, sortindex_, hill_ptrs, hill_data)
                    
                    logging.info('Report complete.')

                elif datatype == 'bruker':
                    logging.info('Feature finding on {}'.format(file_name))
                    feature_path = extract_bruker(file_name)
                    feature_table = convert_bruker(feature_path)
                    logging.info('Bruker featurer finder complete. Extracted {:,} features.'.format(len(feature_table)))

                logging.info('Matching features to query data.')
                features = map_ms2(feature_table, query_data, **settings['features'])

                logging.info('Saving feature table.')
                ms_file.write(feature_table, dataset_name="feature_table")
                logging.info('Feature table saved to {}'.format(out_file))


            logging.info('Saving features.')
            ms_file.write(features, dataset_name="features")
            logging.info(f'Feature finding of file {file_name} complete.')
        return True
    except Exception as e:
        logging.error(f'Feature finding of file {file_name} failed. Exception {e}')
        return f"{e}" #Can't return exception object, cast as string  

## Mapping

Mapping MS1 to MS2

In [ ]:
#export

from sklearn.neighbors import KDTree
import pandas as pd
import numpy as np

def map_ms2(feature_table, query_data, ppm_range = 20, rt_range = 0.5, mob_range = 0.3, n_neighbors=5, search_unidentified = False, **kwargs):
    """
    Map MS1 features to MS2 based on rt and mz
    if ccs is included also add
    """

    if 'mobility' in feature_table.columns:
        use_mob = True
    else:
        use_mob = False

    if use_mob:

        tree_points = feature_table[['mz','rt_apex','mobility']].values

        tree_points[:,0] = np.log(tree_points[:,0])*1e6/ppm_range #m/z -> log transform, this is in ppm then
        tree_points[:,1] = tree_points[:,1]/rt_range # -> this is in minutes
        tree_points[:,2] = tree_points[:,2]/mob_range

        matching_tree = KDTree(tree_points, metric="minkowski")

        query_mz = np.log(query_data['mono_mzs2'])*1e6/ppm_range
        query_rt = query_data['rt_list_ms2'] / rt_range
        query_mob = query_data['mobility'] / mob_range

        ref_points = np.array([query_mz, query_rt, query_mob]).T

        ref_points[ref_points == -np.inf] = 0
        ref_points[ref_points == np.inf] = 0
        ref_points[np.isnan(ref_points)] = 0

        dist, idx = matching_tree.query(ref_points, k=n_neighbors)

    else:
        tree_points = feature_table[['mz','rt_apex']].values
        tree_points[:,0] = np.log(tree_points[:,0])*1e6/ppm_range #m/z -> log transform, this is in ppm then
        tree_points[:,1] = tree_points[:,1]/rt_range # -> this is in minutes

        matching_tree = KDTree(tree_points, metric="minkowski")

        query_mz = np.log(query_data['mono_mzs2'])*1e6/ppm_range
        query_rt = query_data['rt_list_ms2'] / rt_range

        ref_points = np.array([query_mz, query_rt]).T

        ref_points[ref_points == -np.inf] = 0
        ref_points[ref_points == np.inf] = 0
        ref_points[np.isnan(ref_points)] = 0

        dist, idx = matching_tree.query(ref_points, k=n_neighbors)


    ref_matched = np.zeros(ref_points.shape[0], dtype=np.bool_)

    all_df = []
    for neighbor in range(n_neighbors):
        if use_mob:
            ref_df = pd.DataFrame(np.array([query_data['rt_list_ms2'], query_data['prec_mass_list2'], query_data['mono_mzs2'], query_data['charge2'], query_data['mobility']]).T, columns=['rt', 'mass', 'mz', 'charge','mobility'])
        else:
            ref_df = pd.DataFrame(np.array([query_data['rt_list_ms2'], query_data['prec_mass_list2'], query_data['mono_mzs2'], query_data['charge2']]).T, columns=['rt', 'mass', 'mz', 'charge'])

        ref_df['mass_matched'] = feature_table.iloc[idx[:,neighbor]]['mass'].values
        ref_df['mass_offset'] = ref_df['mass_matched'] - ref_df['mass']

        ref_df['rt_matched'] = feature_table.iloc[idx[:,neighbor]]['rt_apex'].values
        ref_df['rt_offset'] = ref_df['rt_matched'] - ref_df['rt']

        ref_df['mz_matched'] = feature_table.iloc[idx[:,neighbor]]['mz'].values
        ref_df['mz_offset'] = ref_df['mz_matched'] - ref_df['mz']

        if use_mob:
            ref_df['mobility_matched'] = feature_table.iloc[idx[:,neighbor]]['mobility'].values
            ref_df['mobility_offset'] = ref_df['mobility_matched'] - ref_df['mobility']

        ref_df['charge_matched'] = feature_table.iloc[idx[:,neighbor]]['charge'].values

        ref_df['query_idx'] = ref_df.index
        ref_df['feature_idx'] = idx[:,neighbor]

        for field in ['int_sum','int_apex','rt_start','rt_apex','rt_end','fwhm','mobility_lower','mobility_upper']:
            if field in feature_table.keys():
                ref_df[field] = feature_table.iloc[idx[:,neighbor]][field].values
        
        rt_check = (ref_df['rt_start'] <= ref_df['rt']) & (ref_df['rt'] <= ref_df['rt_end'])

        # check isolation window (win=3)
        mass_check = np.abs(ref_df['mz_offset'].values) <= 3

        _check = rt_check & mass_check
        if use_mob:
            mob_check = (ref_df['mobility_lower'] <= ref_df['mobility']) & (ref_df['mobility'] <= ref_df['mobility_upper'])
            _check &= mob_check

        ref_matched |= _check
        ref_df['dist'] = dist[:,neighbor]
        ref_df = ref_df[_check]
    
        #ref_df['dist'] = dist[:,neighbor]
        #ref_matched |= (ref_df['dist']<1)
        #ref_df = ref_df[ref_df['dist']<1]

        all_df.append(ref_df)

    
    if search_unidentified:
        if use_mob:
            unmatched_ref = pd.DataFrame(np.array([query_data['rt_list_ms2'], query_data['prec_mass_list2'], query_data['mono_mzs2'], query_data['charge2'], query_data['mobility']]).T, columns=['rt', 'mass', 'mz', 'charge','mobility'])
        else:
            unmatched_ref = pd.DataFrame(np.array([query_data['rt_list_ms2'], query_data['prec_mass_list2'], query_data['mono_mzs2'], query_data['charge2']]).T, columns=['rt', 'mass', 'mz', 'charge'])
        unmatched_ref = unmatched_ref[~ref_matched]
        unmatched_ref['mass_matched'] = unmatched_ref['mass']
        unmatched_ref['mass_offset'] = 0
        unmatched_ref['rt_matched'] = unmatched_ref['rt']
        unmatched_ref['rt_offset']  = 0
        unmatched_ref['mz_matched'] = unmatched_ref['mz']
        unmatched_ref['mz_offset'] = 0
        unmatched_ref['charge_matched'] = unmatched_ref['charge']
        unmatched_ref['query_idx'] = unmatched_ref.index
        unmatched_ref['feature_idx'] = np.nan

        if use_mob:
            ref_df['mobility_matched'] = unmatched_ref['mobility']
            ref_df['mobility_offset'] = np.nan

        for field in ['int_sum','int_apex','rt_start','rt_apex','rt_end','fwhm']:
            if field in feature_table.keys():
                unmatched_ref[field] = np.nan
        unmatched_ref['dist'] = np.nan

        all_df.append(unmatched_ref)

    features = pd.concat(all_df)

    features = features.sort_values('mass_matched', ascending=True)
    features = features.reset_index(drop=True)

    return features

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_settings.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_interface.ipynb.
Converted 12_speed.ipynb.
Converted index.ipynb.
